Mehrere Major Updates zwingen mich dazu die Anleitung zu ergänzen:


Seafile Server 3.1.6

Centos 7

<!-- TEASER_END -->

Die Installation von Centos 7 erfolgt im Prinzip wie gewohnt. Ist etwas bunter aber grundsätzlich gleich, *ABER* Centos 7 ist jetzt `systemctl` basiert. Bei der Installation vom Apache 2.4.x fällt ausserdem auf, dass es kein rpm-Paket mehr für fastcgi gibt!

Daher wird der Seafile Server jetzt via mod_proxy angesprochen!

Seafile Installation
---

Alles wie gehabt!
Es gibt jetzt auch eine deutschsprachige Webseite zu Seafile! <https://www.seafile-server.de>

Interessant wird es eigentlich erst beim Startup:

Centos 7 ist ist jetzt `systemctl` basiert. Dadurch ist die Startup Prozedur in 2 Scripte aufgeteilt:

`/usr/lib/systemd/scripts/seafile-server`


    #!/bin/bash
    #
    # description: start seafile-server

    user=seafile
    script_path=/srv/seafile/b9i/seafile-server-latest

    start() {
        sudo -u ${user} ${script_path}/seafile.sh start > /tmp/seafile.init.log 2>&1
        sudo -u ${user} ${script_path}/seahub.sh start > /tmp/seahub.init.log 2>&1

        return $RETVAL
    }


    stop() {
        sudo -u ${user} ${script_path}/seafile.sh stop
        sudo -u ${user} ${script_path}/seahub.sh stop
    }

    # See how we were called.
    case "$1" in
    start)
        start
        ;;
    stop)
        stop
        ;;
    restart|force-reload)
        stop
        start
        ;;
    *)
        echo $"Usage: $0 {start|stop|restart|force-reload}"
        exit 2
    esac

`/usr/lib/systemd/system/seafile.service`

    [Unit]
    Description=Seafile Server Startup

    [Service]
    Type=forking
    ExecStart=/usr/lib/systemd/scripts/seafile-server start
    ExecStop=/usr/lib/systemd/scripts/seafile-server stop

    [Install]
    WantedBy=multi-user.target

Dann kann Seafile via 
`systemctl start seafile.service`
gestartet werden. Gestoppt wird es via
`systemctl stop seafile.service`

Und am wichtigsten: `systemctl status seafile.service`

Eine sehr gute Einweisung zu systemctl habe ich auf folgender Webseite gefunden: <http://www.cromwell-intl.com/linux/linux-boot.html#systemd>

Sudo anpassen
---

Da von systemctl das Script ohne pseudotty gestartet wird, bricht der sudo Befehl ab!
bitte folgendes aus mit `visudo` in der Konfig von sudo auskommentieren:

    #Defaults    requiretty

httpd anpassen (Apache Version 2.4.x)
---

Die folgenden Anweissungen müssen in die Konfigdatei vom Apache eingefugt werden. Da bei mir der Seafile Server in einer VirtualHost Umgebung läuft, zwischen:
`<VirtualHost *:80> --- </VirtualHost>`

    RewriteEngine On

    #
    # seafile httpserver
    #
      ProxyPass /seafhttp http://127.0.0.1:8082 retry=0
      ProxyPassReverse /seafhttp http://127.0.0.1:8082
      RewriteRule ^/seafhttp - [QSA,L]


    #
    # seafile webdav
    #
      ProxyPassReverse /seafdav http://127.0.0.1:8080

    #
    # seahub
    #
      ProxyPass / http://127.0.0.1:8000/
      ProxyPassReverse / http://127.0.0.1:8000
      RewriteRule ^/(media.*)$ /$1 [QSA,L,PT]

Diese Konfiguration läuft bei mir sehr gut und stabil. Ich vermisse daher mod_fcgid nicht! 

Um Vorschläge wie man die "init" Scripte von `systemctl` besser machen kann wäre ich sehr dankbar.